In [75]:
from bs4 import BeautifulSoup
import requests
import re
import pandas

In [35]:
url = "https://lookup.boe.ohio.gov/vtrapp/athens/avlookup.aspx"

In [36]:
 #Get the original page
response = requests.get(url)

In [43]:
response_soup = BeautifulSoup(response.content)

In [44]:
# curl 'https://lookup.boe.ohio.gov/vtrapp/athens/avlookup.aspx' \
#   -H 'authority: lookup.boe.ohio.gov' \
#   -H 'cache-control: max-age=0' \
#   -H 'origin: https://lookup.boe.ohio.gov' \
#   -H 'upgrade-insecure-requests: 1' \
#   -H 'dnt: 1' \
#   -H 'content-type: application/x-www-form-urlencoded' \
#   -H 'user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36' \
#   -H 'accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9' \
#   -H 'sec-fetch-site: same-origin' \
#   -H 'sec-fetch-mode: navigate' \
#   -H 'sec-fetch-user: ?1' \
#   -H 'sec-fetch-dest: document' \
#   -H 'referer: https://lookup.boe.ohio.gov/vtrapp/athens/avlookup.aspx' \
#   -H 'accept-language: en-US,en;q=0.9' \
#   -H 'cookie: _ga=GA1.2.1662054538.1572970731; __cfduid=d1db2dc7146ccac0c64541b5d5540b7fc1604168643; _gid=GA1.2.886396333.1604168645; _gat_gtag_UA_127176238_3=1' \
#   --data-raw '__LASTFOCUS=&__EVENTTARGET=&__EVENTARGUMENT=
#    &__VIEWSTATE=...
#    &__VIEWSTATEGENERATOR=B6786635
#    &__EVENTVALIDATION=...
#    &cmbelectionlist=20201103G
#    &frmLname=Lachman
#    &frmfname=Benjamin
#    &btnsubmit=Submit
#    &txt_byear=&txt_bmvid=&hid_byear=&hid_bmvid=&hid_app=' \
#   --compressed

In [78]:
#Scrape the state data we need to validate the form request
def get_status(fname, lname, response):
    viewstate = response.find('input' , id ='__VIEWSTATE')['value']
    eventvalidation=response.find('input' , id ='__EVENTVALIDATION')['value']
    viewstategenerator=response.find('input' , id ='__VIEWSTATEGENERATOR')['value']
    params={'__EVENTTARGET':'','__EVENTARGUMENT':'','__VIEWSTATE':viewstate,
            '__VIEWSTATEGENERATOR':viewstategenerator,
            '__EVENTVALIDATION':eventvalidation,
            'cmbelectionlist':'20201103G',
            'frmLname':lname,
            'frmfname':fname,
            'btnsubmit':'Submit'}

    #Use the validation data when making the request for all current applications
    r = requests.post(url, data=params)
    soup = BeautifulSoup(r.text, "html.parser")
    
    results = next(iter(soup.select('#numresults p')), None)
    regex = r'Total Records found:\s?(\d+)'
    
    if results is None:
        return
    
    match = re.search(regex,results.text)
    
    if match.group(1) == '0':
        print('no matches')
        return
    
    lines = next(iter(soup.select('#tblvtrlist p')), None).text.split('\n')
    print([y for y in map(lambda x: x.strip(), lines) if len(y) != 0])

#     for tag in soup.find_all('td'):
#         print(tag)

In [77]:
df = pandas.read_csv('csv/athens-absentee_list-2020_08_13-final.csv')

for index, row in df[:30].iterrows():
    print(f'{row["FIRSTN"]} {row["MIDDLEN"]} {row["LASTN"]}')       
    get_status(fname=row['FIRSTN'], lname=row['LASTN'], response=response_soup)       

HANNAH MARIE ADKINS
['Date Approved for Counting: 10/13/2020', 'Your ballot has been received and will be included in the election night results.']
JAMES M ALLEN
no matches
ALIA A ANDERSON
no matches
COLLIN ANDREW AROCHO
[]
EMILY N BAILS
['Date Approved for Counting: 10/21/2020', 'Your ballot has been received and will be included in the election night results.']
DONNA J BARNHART
no matches
NORMA JEAN BARTLETT
no matches
ROGER LEE BARTLETT
no matches
ANGELICA LEE BELL
no matches
KARA DAWNE BOLIN
no matches
LEWIS G BREWSTER
no matches
STANLEY WAYNE BROWN
no matches
GERARD M BURKE
['Date Approved for Counting: 10/23/2020', 'Your ballot has been received and will be included in the election night results.']
ELLA RACHAEL BUSCH
['Date Approved for Counting: 10/16/2020', 'Your ballot has been received and will be included in the election night results.']
JENNIFER KAYE CAIN
no matches
JEREMY ALAM CAIN
no matches
CLIFFORD LEON CANTER
no matches
GRIFFIN E CHMIEL
['Date Approved for Counting: 10